In [190]:
import numpy as np
from twitter_scraper import get_tweets
# from twitterscraper import query_tweets
import re
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt

In [191]:
person_of_interest = 'realDonaldTrump'
# tweets_info = query_tweets('from:' + person_of_interest, begindate=dt.date(2018,4,1), enddate=dt.date.today())
# #     # def query_tweets(query, limit=None, begindate=dt.date(2017,1,1), enddate=dt.date.today(), poolsize=20, lang=''):
    
# # tweets = []
# # for tweet_info in tweets_info:
# #     tweet = tweet_info.text
# #     tweets.append(tweet)

# # tweets = np.array(tweets)

# # print(tweets)
# print(tweets_info)

In [192]:
tweets_info = get_tweets(person_of_interest, pages = 10)
tweets = []
for tweet_info in list(tweets_info):
    text = tweet_info['text']
#     text = tweet_info.text #Figure out which one to use. It does not work consistently yet
    # Add NEWTWEET to help recognize starting word of tweet for future processing
    tweets.append("NEWTWEET " + text)
    
tweets.append("NEWTWEET") # Ensures that last word in last tweet is tied to end state
    
tweets = np.array(tweets)
num_tweets = len(tweets)
print("Number of tweets: " + str(num_tweets))
# print(tweets)

Number of tweets: 201


In [193]:
words = []
for tweet in tweets:
    words.extend(tweet.split(" "))
    
#edge case of URL attatched to end of sentence without space: ... American families.img.pic.twitter...
cleaned_words = []
for word in words:
    url = [".com", "www.", "http", "https://"]
    if all(x not in word for x in url) and word != '':
#     if ".com" not in word and "www." not in word and "http" not in word and word != '': 
        cleaned_words.append(word)
#     bonuses!pic.twitter.com/0u3dpLJiV2
    elif "pic." in word:
        index_of_url = word.index("pic.")
        cleaned_words.append(word[:index_of_url])
        
words = cleaned_words
cleaned_words_no_repeat = [words[0]] # could be repeated 'NEWTWEET' due to deleting URL's
        
for i in range(1, len(words)):
    if not (words[i] == "NEWTWEET" and words[i] == words[i-1]):
        cleaned_words_no_repeat.append(words[i])
        
words = cleaned_words_no_repeat
# print(words)
# print(words.index("and"))
words = np.array(words)

In [194]:
G = nx.DiGraph()
# Initialize states with start and end states (assuming no word is 'S' or 'E')
states = ['S', 'E']
for word in words:
    if word not in states and word != "" and word != "NEWTWEET":
        states.append(word)
G.add_nodes_from(states)
# nx.draw(G, with_labels = True)
# print(states)

In [195]:
freq = {}
edges = []

# Edge case for first word in tweet not being connected to Start state

for i in range(len(words) - 1):
    if words[i] == "NEWTWEET":
        edges.append(('S', words[i + 1], 1))
        continue
    elif words[i+1] == "NEWTWEET":
        edges.append((words[i], 'E', 1))
        continue
    elif (words[i], words[i+1]) in freq:
        freq[(words[i], words[i+1])] += 1
    else:
        freq[(words[i], words[i+1])] = 1
        
    if words[i].endswith(('.', '!', '?')):
        edges.append((words[i], 'E', 1))
        edges.append(('S', words[i + 1], 1))
        
edges.append(('E', 'S', 1))

for edge in freq:
    edges.append((edge[0], edge[1], freq[edge]))
    
G.add_weighted_edges_from(edges)
# nx.draw(G, with_labels = True)
# nx.draw(G)

In [196]:
# print(list(G.successors('S')))
# print(list(G.predecessors('E')))

# print(list(G.predecessors('S')))
# print(list(G.successors('E')))

In [197]:
# Creates numpy adjacency matrix of states, where Starting node is row 0 and Ending node is row 1
matrix = nx.to_numpy_matrix(G, nodelist = states)
matrix = np.array(matrix).tolist()
# print(matrix)
# print(np.array(matrix))

In [198]:
# Sanity check to see if adjacency matrix reflects actual state of MC
num = 0
for i in range(len(matrix[0])):
    num += matrix[1][i]
print(num == len(list(G.successors('E')))) 

True


In [199]:
for i in range(len(matrix)):
    if np.array(matrix[i]).sum(axis=0) != 0:
        continue
    else:
        print(i, np.array(matrix[i]).sum(axis=0))
        print('broken state: ' + states[i])
        print(list(G.predecessors(states[i])))

matrix = np.delete(matrix, len(matrix)-1, axis=1)
matrix = np.delete(matrix, len(matrix)-1, axis=0)

In [200]:
matrix = np.array(matrix)

transition_mat = np.array(matrix)/np.array(matrix).sum(axis=1)[:,None]

def get_next_state(i, transition_matrix):
    return np.random.choice(transition_matrix.shape[1], 1, p=transition_matrix[i])[0]

def perform_random_walk(s, e, transition_matrix):
    curr_state = s
    l = []
    while curr_state != e:
        curr_state = get_next_state(curr_state, transition_matrix)
        if curr_state != s:
            l.append(curr_state)
    return np.array(l)

def generate_sentence():
    return ''.join([states[i] + ' ' for i in perform_random_walk(0, 1, transition_mat) if i != 1])

In [201]:
tweet = generate_sentence()
if tweet.endswith('..'):
    tweet += ' ' + generate(sentence)
elif tweet.startswith('..'):
    tweet = generate_sentence() + ' ' + tweet
print(tweet)

We are responsible for hard working American PRIDE! 


In [253]:
def guided_generation(w1, w2, transition_mat):
    """
    Choose w1, w2 such that they are in the data set
    """
    w1 = states.index(w1)
    w2 = states.index(w2)
    
    mat1 = np.copy(transition_mat)
    mat2 = np.copy(transition_mat)
    
    for i in range(mat1.shape[0]):
        mat1[i][w1] += 0.05
        mat2[i][w2] += 0.05
    
    mat1 = mat1/mat1.sum(axis=1)[:,None]
    mat2 = mat2/mat2.sum(axis=1)[:,None]
    
    start = [states[i] + ' ' for i in perform_random_walk(0, w1, mat1) if i != 1]
    middle = [states[i] + ' ' for i in perform_random_walk(w1, w2, mat2) if i != 1]
    end = [states[i] + ' ' for i in perform_random_walk(w1, 1, transition_mat) if i != 1]
    
    
    return ''.join(start + middle + end)
    

In [254]:

# print(list(G.predecessors('and')))
# print(list(G.successors('and')))

In [255]:
tweet = guided_generation("Obama", "BAD", transition_mat)
print(tweet)

...Congress must immediately pass a great economic power, is considered a Obama BAD joke. 


In [264]:
tweet = guided_generation("Korea", "WALL", transition_mat)
print(tweet)

This is a Korea would be WALL last night from China, there was NO WAY. 


In [274]:
tweet = guided_generation("FBI", "golf", transition_mat)
print(tweet)

Problem is that we have been a crime infested & Russia. Obama at the Supreme Court! AUSTIN BOMBING SUSPECT IS DEAD. FBI (why didn’t care. I golf (why didn’t have sought review. 
